In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

/home/faiz/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
df=pd.read_csv("cal_housing_clean.csv")

In [4]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [5]:
from sklearn.model_selection import train_test_split

In [7]:
x_data=df.drop(["medianHouseValue"],axis=1)

In [9]:
y_true=df["medianHouseValue"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_true,test_size=0.3,random_state=101)

# Scale Feature Data With Sklearn Preprocessing

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
scaler=MinMaxScaler()

In [15]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [16]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [17]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

# Create Feature Columns

In [19]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [20]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [21]:
feat_col =[ age,rooms,bedrooms,pop,households,income]

In [23]:
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


In [24]:
X_test.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16086,0.686275,0.046264,0.045158,0.025873,0.048841,0.353133
8816,0.705882,0.027417,0.020795,0.012709,0.023187,0.770182
7175,0.901961,0.032326,0.040813,0.041662,0.042592,0.133626
16714,0.313725,0.043212,0.046089,0.032840,0.048018,0.263576
14491,0.411765,0.088433,0.069367,0.043728,0.072192,0.660046


In [26]:
y_test.head()

16086    406200.0
8816     500001.0
7175     122900.0
16714    209100.0
14491    500001.0
Name: medianHouseValue, dtype: float64

# Create the input function for the estimator object. 

In [28]:
inputfunc=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

# Create the estimator model. Use a DNNRegressor.

In [31]:
model=tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_col)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9zta2pc8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa39ea34f60>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [33]:
model.train(input_fn=inputfunc,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp9zta2pc8/model.ckpt.
INFO:tensorflow:loss = 448984120000.0, step = 1
INFO:tensorflow:global_step/sec: 335.926
INFO:tensorflow:loss = 780863000000.0, step = 101 (0.299 sec)
INFO:tensorflow:global_step/sec: 420.536
INFO:tensorflow:loss = 394878500000.0, step = 201 (0.243 sec)
INFO:tensorflow:global_step/sec: 419.068
INFO:tensorflow:loss = 547217570000.0, step = 301 (0.234 sec)
INFO:tensorflow:global_step/sec: 416.353
INFO:tensorflow:loss = 841429400000.0, step = 401 (0.242 sec)
INFO:tensorflow:global_step/sec: 431.392
INFO:tensorflow:loss = 432708520000.0, step = 501 (0.229 sec)
INFO:tensorflow:global_step/sec: 383.102
INFO:tensorflow:loss = 443856260000.0, step = 601 (0.266 sec)
INFO:tensorflow:global_step/sec: 387.542
INFO:tensorflow:loss = 633741200000.0, step = 701 (0.253 sec)
INFO:tensorflow:global_step/sec: 425.885
INFO:tensorflow:loss = 623031350000.0, step = 801 (0.237 sec)
INFO:tens

INFO:tensorflow:global_step/sec: 421.075
INFO:tensorflow:loss = 54624310000.0, step = 7901 (0.236 sec)
INFO:tensorflow:global_step/sec: 435.906
INFO:tensorflow:loss = 42237407000.0, step = 8001 (0.230 sec)
INFO:tensorflow:global_step/sec: 371.982
INFO:tensorflow:loss = 58013573000.0, step = 8101 (0.268 sec)
INFO:tensorflow:global_step/sec: 405.774
INFO:tensorflow:loss = 87034640000.0, step = 8201 (0.249 sec)
INFO:tensorflow:global_step/sec: 343.967
INFO:tensorflow:loss = 135483100000.0, step = 8301 (0.294 sec)
INFO:tensorflow:global_step/sec: 386.397
INFO:tensorflow:loss = 154846860000.0, step = 8401 (0.260 sec)
INFO:tensorflow:global_step/sec: 332.512
INFO:tensorflow:loss = 167245580000.0, step = 8501 (0.294 sec)
INFO:tensorflow:global_step/sec: 304.457
INFO:tensorflow:loss = 78915430000.0, step = 8601 (0.333 sec)
INFO:tensorflow:global_step/sec: 239.361
INFO:tensorflow:loss = 145909420000.0, step = 8701 (0.416 sec)
INFO:tensorflow:global_step/sec: 281.942
INFO:tensorflow:loss = 12585

INFO:tensorflow:global_step/sec: 316.302
INFO:tensorflow:loss = 141877360000.0, step = 15801 (0.312 sec)
INFO:tensorflow:global_step/sec: 331.355
INFO:tensorflow:loss = 97205460000.0, step = 15901 (0.308 sec)
INFO:tensorflow:global_step/sec: 347.22
INFO:tensorflow:loss = 39257956000.0, step = 16001 (0.283 sec)
INFO:tensorflow:global_step/sec: 308.678
INFO:tensorflow:loss = 148695800000.0, step = 16101 (0.337 sec)
INFO:tensorflow:global_step/sec: 275.593
INFO:tensorflow:loss = 93812820000.0, step = 16201 (0.345 sec)
INFO:tensorflow:global_step/sec: 355.052
INFO:tensorflow:loss = 61332340000.0, step = 16301 (0.286 sec)
INFO:tensorflow:global_step/sec: 332.918
INFO:tensorflow:loss = 196853100000.0, step = 16401 (0.300 sec)
INFO:tensorflow:global_step/sec: 364.284
INFO:tensorflow:loss = 44909930000.0, step = 16501 (0.279 sec)
INFO:tensorflow:global_step/sec: 234.018
INFO:tensorflow:loss = 46294307000.0, step = 16601 (0.425 sec)
INFO:tensorflow:global_step/sec: 376.797
INFO:tensorflow:loss 

INFO:tensorflow:global_step/sec: 360.82
INFO:tensorflow:loss = 80469230000.0, step = 23701 (0.277 sec)
INFO:tensorflow:global_step/sec: 351.907
INFO:tensorflow:loss = 152531800000.0, step = 23801 (0.282 sec)
INFO:tensorflow:global_step/sec: 352.641
INFO:tensorflow:loss = 61294023000.0, step = 23901 (0.283 sec)
INFO:tensorflow:global_step/sec: 374.768
INFO:tensorflow:loss = 58046915000.0, step = 24001 (0.267 sec)
INFO:tensorflow:global_step/sec: 359.779
INFO:tensorflow:loss = 94185520000.0, step = 24101 (0.277 sec)
INFO:tensorflow:global_step/sec: 360.399
INFO:tensorflow:loss = 96083804000.0, step = 24201 (0.280 sec)
INFO:tensorflow:global_step/sec: 343.575
INFO:tensorflow:loss = 107107770000.0, step = 24301 (0.289 sec)
INFO:tensorflow:global_step/sec: 362
INFO:tensorflow:loss = 107337160000.0, step = 24401 (0.280 sec)
INFO:tensorflow:global_step/sec: 366.047
INFO:tensorflow:loss = 49992940000.0, step = 24501 (0.269 sec)
INFO:tensorflow:global_step/sec: 361.504
INFO:tensorflow:loss = 80

# Create a prediction input

In [34]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [35]:
pred_gen = model.predict(predict_input_func)

In [36]:
pred_gen

<generator object Estimator.predict at 0x7fa352502780>

In [37]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /tmp/tmp9zta2pc8/model.ckpt-25000


In [38]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [39]:
final_preds

[array([233416.67], dtype=float32),
 array([294253.], dtype=float32),
 array([217287.22], dtype=float32),
 array([185230.92], dtype=float32),
 array([266220.4], dtype=float32),
 array([201662.89], dtype=float32),
 array([228166.8], dtype=float32),
 array([207308.47], dtype=float32),
 array([215270.66], dtype=float32),
 array([185092.92], dtype=float32),
 array([206821.72], dtype=float32),
 array([226821.84], dtype=float32),
 array([194046.53], dtype=float32),
 array([179019.25], dtype=float32),
 array([259498.52], dtype=float32),
 array([176836.1], dtype=float32),
 array([202850.03], dtype=float32),
 array([187586.44], dtype=float32),
 array([181990.77], dtype=float32),
 array([232618.23], dtype=float32),
 array([172613.1], dtype=float32),
 array([234785.94], dtype=float32),
 array([216455.06], dtype=float32),
 array([170711.61], dtype=float32),
 array([191907.4], dtype=float32),
 array([262451.88], dtype=float32),
 array([187502.64], dtype=float32),
 array([149005.61], dtype=float32),

In [40]:
from sklearn.metrics import mean_squared_error

In [41]:
mean_squared_error(y_test,final_preds)**0.5

101502.7281916986